# Sentiment Analysis con KNN
## Clasificador en C++ 💪💪
Vamos a probar a nuestro bichito


Definir los path al ejecutable de python 3.6 y sus librerías,
de acuerdo al virtual env que estén corriendo.

In [1]:
!cd .. && git submodule init
!cd .. && git submodule update
!cd .. && mkdir build
!cd ../build/ && rm -rf *
!cd ../build && cmake \
  -DPYTHON_EXECUTABLE="$(which python)" \
  -DCMAKE_BUILD_TYPE=Release ..
!cd ../build && make install


mkdir: cannot create directory ‘build’: File exists
-- The C compiler identification is GNU 5.5.0
-- The CXX compiler identification is GNU 5.5.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
Release mode
-- Found PythonInterp: /home/luis/miniconda3/bin/python (found version "3.6.5") 
-- Found PythonLibs: /home/luis/miniconda3/lib/libpython3.6m.so
-- pybind11 v2.3.dev0
-- Performing Test HAS_FLTO
-- Performing Test HAS_FLTO - Success
-- LTO enabled
CMAKE_INSTALL_PREFIX=/home/luis/dev/metodos-tp2/metnum-tp2-20191c/metnum-tp2-20191c
-- Config

In [2]:
# Verifico la correcta instalación. Si no falla el import está OK
!pwd
!python --version
import sentiment
from sentiment import entrenar
from sentiment import generate_vocabulary
from sentiment import entrenar_varios

/home/luis/dev/metodos-tp2/metnum-tp2-20191c/metnum-tp2-20191c/notebooks
Python 3.6.5 :: Anaconda, Inc.


In [3]:
%load_ext autoreload
%autoreload 2
import pandas as pd

!cd ../data && tar -xvf *.tgz

df = pd.read_csv("../data/imdb_small.csv", index_col=0)

print("Cantidad de documentos: {}".format(df.shape[0]))

./._imdb_small.csv
imdb_small.csv
Cantidad de documentos: 12500


In [4]:
df.describe()

,type,review,label,file
count,12500,12500,12500,12500
unique,2,12469,2,12085
top,test,Aside from the horrendous acting and the ridic...,neg,11457_10.txt
freq,6275,2,6322,2


In [4]:
text_train = df[df.type == 'train']["review"]
label_train = df[df.type == 'train']["label"]
text_test = df[df.type == 'test']["review"]
label_test = df[df.type == 'test']["label"]

print("Cantidad de instancias de entrenamiento = {}".format(len(text_train)))
print("Cantidad de instancias de test = {}".format(len(text_test)))


Cantidad de instancias de entrenamiento = 6225
Cantidad de instancias de test = 6275


In [5]:
def particion_criticas_equilibradas(text_train, label_train, cant_elems):
    dict_texto = text_train.to_dict()
    dict_label = label_train.to_dict()
    claves_positivos = [x for x in dict_label if dict_label[x] == 'pos']
    claves_negativos = [x for x in dict_label if dict_label[x] == 'neg']
    posiciones_elegidas_pos = random.sample(range(0, len(claves_positivos)), int(cant_elems/2))
    posiciones_elegidas_neg = random.sample(range(0, len(claves_negativos)), int(cant_elems/2))
    nuevo_text_train = {}
    nuevo_label_train = {}
    for i in posiciones_elegidas_pos:
        nuevo_label_train[claves_positivos[i]] = 'pos'
        nuevo_text_train[claves_positivos[i]] = dict_texto[claves_positivos[i]]
    for i in posiciones_elegidas_neg:
        nuevo_label_train[claves_negativos[i]] = 'neg'
        nuevo_text_train[claves_negativos[i]] = dict_texto[claves_negativos[i]]
    return pd.Series(nuevo_text_train), pd.Series(nuevo_label_train)

In [6]:
print("Class balance : {} pos {} neg".format(
    (label_train == 'pos').sum() / label_train.shape[0], 
    (label_train == 'neg').sum() / label_train.shape[0]
))

Class balance : 0.49493975903614457 pos 0.5050602409638554 neg


In [6]:
from sklearn.feature_extraction.text import CountVectorizer
import random

#vectorizer = CountVectorizer(max_df=0.90, min_df=0.01, max_features=5000)
vectorizer = CountVectorizer(stop_words="english", analyzer='word', max_df=0.60, min_df=0.08, max_features=5000)
voc = vectorizer.fit(text_train)

print("Vocabulary content:\n {}".format(voc.vocabulary_))

criticas_a_experimentar = 100
#text_test, label_test = particion_criticas_equilibradas(text_test,label_test,criticas_a_experimentar)


X_train, y_train = vectorizer.transform(text_train), (label_train == 'pos').values
X_test, y_test = vectorizer.transform(text_test), (label_test == 'pos').values




Vocabulary content:
 {'ll': 60, 'know': 55, 'br': 13, 'just': 53, 'new': 74, 'time': 105, 'right': 88, 'probably': 84, 'worst': 119, 'film': 35, 've': 110, 'seen': 96, 'acting': 1, 'characters': 16, 'looking': 63, 'films': 36, 'really': 87, 'takes': 100, 'make': 67, 'look': 62, 'music': 73, 'scene': 92, 'believe': 8, 'better': 10, 'pretty': 83, 'especially': 30, 'actors': 4, '10': 0, 'director': 24, 'say': 91, 'making': 69, 'don': 27, 'want': 111, 'kind': 54, 'movies': 72, 'woman': 116, 'does': 25, 'like': 58, 'actor': 3, 'character': 15, 'great': 45, 'story': 98, 'far': 33, 'main': 66, 'did': 22, 'wasn': 112, 'old': 75, 'years': 121, 'times': 106, 'end': 28, 'tv': 109, 'sure': 99, 'good': 43, 'man': 70, 'way': 115, 'saw': 90, 'think': 103, 'point': 82, 'people': 77, 'played': 80, 'let': 56, 'role': 89, 'didn': 23, 'got': 44, 'day': 21, 'performance': 78, 'action': 2, 'script': 95, 'plot': 81, 'fact': 31, 'gets': 39, 'minutes': 71, 'doesn': 26, 'little': 59, 'interesting': 51, 'best': 

In [7]:
import sentiment
import numpy as np


clf = sentiment.KNNClassifier(100)#100neigbourds

clf.fit(X_train.toarray(), y_train)
#print(X_train)
#print((clf.vectores() != X_train))
#print((clf.labels() != y_train))


#Vemos que son iguales
for index, (first, second) in enumerate(zip(y_train, clf.labels().astype(bool))):
    if first != second:
        print(index, second)
        



In [8]:
%%time
from sklearn.metrics import accuracy_score
import time
start_time = time.clock()
y_pred = clf.predict(X_test.toarray())
print (time.clock() - start_time, "segundos en correr {} criticas".format(len(y_pred)))
acc = accuracy_score(y_test, y_pred.astype(bool))
print("Accuracy: {}".format(acc))

14.670628 segundos en correr 6275 criticas
Accuracy: 0.6873306772908366
CPU times: user 14.6 s, sys: 67.6 ms, total: 14.7 s
Wall time: 14.6 s


In [10]:
import sentiment
import numpy as np
clf2 = sentiment.PCA(10)
clf2.fit(X_train.toarray())



M = (X_train - X_train.mean(axis=0))
cov_matrix = M.T @ M / (M.shape[0]-1) 

print(cov_matrix.shape[0], cov_matrix.shape[1])
print(clf2.cov().shape[0], clf2.cov().shape[1])
#print(M[0, :])
#print(cov_matrix[0,:])
#print(clf2.cov()[0, :])


<class 'scipy.sparse.csr.csr_matrix'>
<class 'scipy.sparse.csr.csr_matrix'>
123 123
123 123


In [10]:
%%time
from sklearn.metrics import accuracy_score

clf2 = sentiment.PCA(10) #10 es la alfa, la cantidad de autovectores/valores que usamos
#Si X_train es de dimension n x m, n  siendo instancias y m cantidad de variables
#V matriz cambio de base va a ser de dimension m x alfa


clf2.fit(X_train.toarray())#Genera y guarda matriz cambiodeBase de X_train, a partir de la matriz de covarianza.

X_pda_train = clf2.transform(X_train)#Hace X_train = X_train *V(cambiodeBase generado por el fit)
#La matriz resultante que vamos a usar en el knn tiende dimension n x alfa

#Como cambiamos de base, tenemos que cambiar la base de las cosas en test para que vivan en el mismo espacio

X_pda_test = clf2.transform(X_test)

#dimension de X_test luego de transform = n x alfa.

#Luego se sigue normalmente

clf3 = sentiment.KNNClassifier(100) #hago un clf nuevo para no mezclar

clf3.fit(X_pda_train, y_train)

y_pred = clf3.predict(X_pda_test[0:10])

acc = accuracy_score(y_test[0:10], y_pred.astype(bool))
print("Accuracy: {}".format(acc))





Accuracy: 0.7
CPU times: user 2.82 s, sys: 12 ms, total: 2.84 s
Wall time: 2.83 s
